# This notebook is for the KNN model


In [20]:
import pandas as pd
import numpy as np
import qgrid
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from modAL.models import ActiveLearner

Set up Training Data

In [11]:
#All amines except for dimethylammonium iodide
viable_amines = ['ZEVRFFCPALTVDN-UHFFFAOYSA-N',
                  'KFQARYBEAKAXIC-UHFFFAOYSA-N',
                  'NLJDBTZLVTWXRG-UHFFFAOYSA-N',
                  'LCTUISCIGMWMAT-UHFFFAOYSA-N',
                  'JERSPYRKVMAEJY-UHFFFAOYSA-N',
                  'VAWHFUNJDMQUSB-UHFFFAOYSA-N',
                  'WGYRINYTHSORGH-UHFFFAOYSA-N',
                  'FCTHQYIDLRRROX-UHFFFAOYSA-N',
                  'VNAAUNTYIONOHR-UHFFFAOYSA-N',
                  'KOAGKPNEVYEZDU-UHFFFAOYSA-N',
                  'FJFIJIDZQADKEE-UHFFFAOYSA-N',
                  'XFYICZOIWSBQSK-UHFFFAOYSA-N',
                  'KFXBDBPOGBBVMC-UHFFFAOYSA-N',
                  'HBPSMMXRESDUSG-UHFFFAOYSA-N',
                  'NXRUEVJQMBGVAT-UHFFFAOYSA-N',
                  'CALQKRVFTWDYDG-UHFFFAOYSA-N',
                  'LLWRXQXPJMPHLR-UHFFFAOYSA-N',
                  'BAMDIFIROXTEEM-UHFFFAOYSA-N',
                  'XZUCBFLUEBDNSJ-UHFFFAOYSA-N']

distribution_header = '_raw_modelname'
amine_header = '_rxn_organic-inchikey'
score_header = '_out_crystalscore'
name_header = 'name'
to_exclude = [score_header, amine_header,
              name_header, distribution_header, '_raw_RelativeHumidity']
bool_cols = ['_solv_GBL',
                  '_solv_DMSO',
                  '_solv_DMF',
                  '_feat_primaryAmine',
                  '_feat_secondaryAmine',
                  '_rxn_plateEdgeQ']

df = pd.read_csv('../data/raw/0057.perovskitedata_DRPFeatures_2020-07-02.csv')
df = df[df[distribution_header].str.contains('Uniform')]
df = df[df[amine_header].isin(viable_amines)]

df[score_header] = [1 if val == 4 else 0 for val in
                         df[score_header].values]
y = df[score_header].values
df = df.drop(to_exclude, axis=1)
binary_cols = df[bool_cols].values
df = df.drop(bool_cols, axis=1).values

scaler = StandardScaler()
std_df = scaler.fit_transform(df)

In [18]:
training_df = np.concatenate((std_df, binary_cols), axis = 1)
training_df.shape

(1626, 50)

In [21]:
knn = KNeighborsClassifier(n_neighbors=1,leaf_size=1, p=1)
learner = ActiveLearner(estimator=knn, X_training=training_df, y_training=y.ravel())

In [36]:
raw_df = pd.read_csv('../data/raw/0057.perovskitedata_redone_2020-07-02.csv', low_memory=False)
raw_df = raw_df[raw_df[distribution_header].str.contains('Uniform')]
raw_df = raw_df[raw_df['_raw_organic_0_inchikey'].isin(['JMXLWMIFDJCGBV-UHFFFAOYSA-N'])]
raw_df.shape

(96, 696)

In [37]:
raw_df.to_csv('raw_dimethyl_finished.csv')

In [38]:
df = pd.read_csv('../data/raw/0057.perovskitedata_DRPFeatures_2020-07-02.csv')
df = df[df[distribution_header].str.contains('Uniform')]
df.to_csv('dimethyl_processed.csv')